In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word_to_index = {}
        self.word_to_count = {}
        self.index_to_word = {0: "SOS", 1: "EOS"}
        self.num_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word_to_index:
            self.word_to_index[word] = self.num_words
            self.word_to_count[word] = 1
            self.index_to_word[self.num_words] = word
            self.num_words += 1
        else:
            self.word_to_count[word] += 1

In [10]:
# Turn a Unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [11]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [12]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return p[1].startswith(eng_prefixes) and len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [29]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.num_words)
    print(output_lang.name, output_lang.num_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je ne suis pas perfectionniste .', 'i m not a perfectionist .']


In [14]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        # Make the encoder a GRU and also make it bidirectional.
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True)

    def forward(self, input, hidden):
        # Get the embeddings and reshape to be (1, 1, -1)
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)

In [15]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttentionDecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size

        self.output_size = output_size
        
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.attention_projection = nn.Linear(self.hidden_size * 2, self.max_length)
        self.output_projection = nn.Linear(self.hidden_size * 3, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.gru = nn.GRU(self.hidden_size, self.hidden_size)

        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)

        embedded = self.dropout(embedded)


        embedded_hidden = torch.cat((embedded[0], hidden[0]), 1)
        

        attention_scores = self.attention_projection(embedded_hidden)

        attention_weights = F.softmax(attention_scores, dim=1)
        #print("attention_weights:", attention_weights.shape)
        # (1, 1, H)

        attention_context = torch.bmm(attention_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        #print("attention_context:", attention_context.shape)

        output = torch.cat((embedded[0], attention_context[0]), 1)
        #print("output_cat:", output.shape)

        output = self.output_projection(output).unsqueeze(0)
        #print("output_projection:", output.shape)

        output = F.relu(output)
        

        output, hidden = self.gru(output, hidden)
        #print("output_after_gru:", output.shape)
        #print("hidden_after_gru:", hidden.shape)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attention_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [31]:

def indexesFromSentence(lang, sentence):
    return [lang.word_to_index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [17]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    encoder_hidden = encoder.initHidden()


    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)


    encoder_outputs = torch.zeros(max_length, encoder.hidden_size * 2, device=device)

    loss = 0


    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
      
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden[:1]


    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    target_length_used = 0

    if use_teacher_forcing:

        
        target_length_used = target_length
        
        for di in range(target_length):
        
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:

        for di in range(target_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)

            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            
      
            loss += criterion(decoder_output, target_tensor[di])
            
      
            target_length_used += 1
            
   
            if topi.item() == EOS_token:
                break


   
    loss.backward()


    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length_used

In [18]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()

    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  
    plot_loss_total = 0 

    # Initialize the encoder and decoder optimizers with the above learning rate
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    # Get n_iters training pairs

    training_pairs = [tensorsFromPair(random.choice(pairs)) for _ in range(n_iters)]
    
    
    criterion = nn.NLLLoss()

    for it in range(1, n_iters + 1):
        training_pair = training_pairs[it - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # Train on the input, target pair
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        # Update the total loss and the plot loss

        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, it / n_iters),
                                         it, it / n_iters * 100, print_loss_avg))

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

            showPlot(plot_losses)

In [20]:
hidden_size = 256
encoder = EncoderRNN(input_lang.num_words, hidden_size).to(device)
decoder = AttentionDecoderRNN(hidden_size, output_lang.num_words).to(device)

trainIters(encoder, decoder, 75000, print_every=5000)

/var/folders/mx/41vmmjnx00z2pm2st2dnvjb40000gn/T/ipykernel_92168/1954892691.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


2m 48s (- 39m 19s) (5000 6%) 3.0623
5m 36s (- 36m 25s) (10000 13%) 2.4090
8m 14s (- 32m 57s) (15000 20%) 2.0953
10m 47s (- 29m 40s) (20000 26%) 1.8315
13m 28s (- 26m 56s) (25000 33%) 1.6164
16m 5s (- 24m 8s) (30000 40%) 1.4577
18m 24s (- 21m 2s) (35000 46%) 1.2884
20m 41s (- 18m 6s) (40000 53%) 1.1921
23m 2s (- 15m 21s) (45000 60%) 1.0787
25m 29s (- 12m 44s) (50000 66%) 1.0126
27m 43s (- 10m 4s) (55000 73%) 0.9104
29m 59s (- 7m 29s) (60000 80%) 0.8248
32m 25s (- 4m 59s) (65000 86%) 0.7410
35m 2s (- 2m 30s) (70000 93%) 0.6680
37m 34s (- 0m 0s) (75000 100%) 0.6343


In [24]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():

        input_tensor = tensorFromSentence(input_lang, sentence)
        
        input_length = input_tensor.size()[0]
        

        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, 2 * encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        encoder_hidden = encoder_hidden[:1] + encoder_hidden[1:]

        decoder_input = torch.tensor([[SOS_token]], device=device)  
        decoder_hidden = encoder_hidden

        
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):

            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            decoder_attentions[di] = decoder_attention.data
 
            topv, topi = decoder_output.data.topk(1)
            
          
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
     
            else:
                decoded_words.append(output_lang.index_to_word[topi.item()])        


            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [28]:
from nltk.translate.bleu_score import sentence_bleu

def evaluateRandomly(encoder, decoder, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        pair = random.choice(pairs)
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])
       
        output_words, _ = evaluate(encoder, decoder, pair[0])
        

        if output_words[-1] == '<EOS>':
            output_words = output_words[:-1]
                            
        output_sentence = ' '.join(output_words)
      
        score = sentence_bleu([pair[1].split()], output_words, weights=(0.5, 0.5))
        
 
        bleu_scores.append(score)
        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')
    print('The mean BLEU score is: ', np.mean(bleu_scores))

In [32]:
evaluateRandomly(encoder, decoder)

/Users/frankchao/opt/anaconda3/envs/ML/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


The mean BLEU score is:  0.7433791250566426
